In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc 
import warnings
import pandas_profiling
import xgboost as xgb
import copy
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
warnings.filterwarnings('ignore')
#最大表示行数の指定（ここでは50行を指定）
pd.set_option('display.max_rows', 1000)
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 1000)
#infoの省略を阻止
pd.options.display.max_info_rows=500
pd.options.display.max_info_columns=500

In [ ]:
# 下位100位の変数を削除
v=['f_262', 'f_80', 'f_0', 'f_40', 'f_191', 'f_27', 'f_85', 'f_109',
       'f_211', 'f_72', 'f_203', 'f_95', 'f_54', 'f_245', 'f_217', 'f_45',
       'f_266', 'f_88', 'f_86', 'f_24', 'f_93', 'f_12', 'f_121', 'f_141',
       'f_189', 'f_1', 'f_253', 'f_113', 'f_140', 'f_161', 'f_70', 'f_167',
       'f_143', 'f_20', 'f_222', 'f_124', 'f_205', 'f_66', 'f_50',
        'f_49', 'f_115', 'f_290', 'f_154', 'f_166', 'f_230',
       'f_103', 'f_17', 'f_199', 'f_9', 'f_258','f_75', 'f_172', 'f_294', 
       'f_81', 'f_56', 'f_201', 'f_171', 'f_279','investment_id',
       'f_152', 'f_207', 'f_36', 'f_51', 'f_192', 'f_291', 'f_89', 'f_228',
       'f_158', 'f_105', 'f_296', 'f_160', 'f_39', 'f_134', 'f_77', 'f_226',
       'f_135', 'f_173', 'f_274', 'f_235', 'f_13', 'f_144', 'f_112', 'f_215',
       'f_277', 'f_260', 'f_242', 'f_239', 'f_227', 'f_91', 'f_218', 'f_76',
       'f_111', 'f_186', 'f_148', 'f_43', 'f_177', 'f_261', 'f_129', 'f_23',
       'f_126', 'f_288',
       # 相関係数 0.75以上の変数を除外
       'f_299', 'f_243', 'f_82', 'f_280', 'f_71', 'f_254', 'f_214', 'f_157', 'f_97',
       'f_106', 'f_168', 'f_212', 'f_61', 'f_246', 'f_240', 'f_234', 'f_59',
       'f_255', 'f_41', 'f_136', 'f_193', 'f_100', 'f_164']
#        # さらに下位の変数を削除
#        'f_293', 'f_195', 'f_156', 'f_114', 'f_133', 'f_94', 'f_252', 'f_194',
#        'f_5', 'f_176', 'f_298', 'f_204', 'f_37', 'f_169', 'f_259', 'f_18','f_102'

In [ ]:
%%time
df_train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')

In [ ]:
features = [f'f_{i}' for i in range(300)]
astype_dict={}
for f in features:
    astype_dict[f]='float16'

df_train=df_train.astype(astype_dict)

In [ ]:
# # 相関係数　0.8以上の変数を除外
# #w=df_train.corr()
# x=w[w > 0.75]
# y=x[w<1].stack()
# z=[]
# for i in y.keys():
#     z.append(i[1])
# print(set(z))

In [ ]:
# ID500以上に限定してみる
df_train=df_train[df_train['time_id']>=500]

In [ ]:
# df_train=df_train.round(decimals = 3)
# df_train.profile_report(minimal=True)

# 学習に使用するデータを設定
x = df_train.drop(['target','row_id','time_id'],axis=1)
x = x.drop(v,axis=1)
y = df_train['target']

del df_train
#x.dtypes
#gc.collect()  

In [ ]:
# # 相関係数のメトリック自作
# def metric_corr(preds, eval_data):
#     y_true = eval_data.get_label()
#     y_true = np.array(y_true)
#     preds  = np.array(preds)
#     y_true = y_true.astype('float64')
    
#     res=np.corrcoef(preds, y_true)
#     return 'metric_corr', res[0][1], True

In [ ]:
# スタッキング

def xg_kf(x,y):
    predicted = copy.copy(y)
    N_SPLITS = 5
    models = []


    kf = KFold(n_splits=N_SPLITS)
    for i, (train_index, valid_index) in enumerate(kf.split(x, y)):
        print('[CV] {}/{}'.format(i+1, N_SPLITS))
        x_train, x_test = x.iloc[train_index],x.iloc[valid_index]
        y_train, y_test = y.iloc[train_index],y.iloc[valid_index]

        data_train_xg = xgb.DMatrix(x_train, label=y_train)
        data_eval_xg = xgb.DMatrix(x_test, label=y_test)

        # LightGBM parameters
        params = {
                "objective": "reg:squarederror",
                "metric": "rmse",
        }

        # モデルの学習
        model_xg = xgb.train(params=params,
                          dtrain=data_train_xg, # トレーニングデータの指定
                          evals=[(data_train_xg, 'train'), (data_eval_xg, 'eval')], # 検証データの指定
                          num_boost_round = 25
                            )

        pred =model_xg.predict(data_eval_xg)
        predicted.iloc[valid_index]=pred
        models.append(model_xg)
    return models,predicted

In [ ]:

def lgb_kf(x,y):
    #x['predict']=predict
    predicted = copy.copy(y)
    N_SPLITS = 5
    models = []


    kf = KFold(n_splits=N_SPLITS)
    for i, (train_index, valid_index) in enumerate(kf.split(x, y)):
        print('[CV] {}/{}'.format(i+1, N_SPLITS))
        x_train, x_test = x.iloc[train_index],x.iloc[valid_index]
        y_train, y_test = y.iloc[train_index],y.iloc[valid_index]

        data_eval_lgb = lgb.Dataset(x_test,y_test) 
        data_train_lgb = lgb.Dataset(x_train,y_train)

        params = {
                "objective": "regression",
                "metric": "rmse",
                "num_leaves": 14,
                "max_depth": 4,
                "feature_fraction": 0.8,
                "subsample_freq": 1,
                "bagging_fraction": 0.7,
                "min_data_in_leaf": 10,
                "learning_rate": 0.2,
                "boosting": "gbdt",
                "lambda_l1": 0.4,
                "lambda_l2": 0.4,
                "verbosity": -1,
                "random_state": 42,
                'num_iterations': 250,
                'early_stopping_round': 20,
        }

        # モデルの学習
        model_lgb = lgb.train(params,
                          data_train_lgb, # トレーニングデータの指定
                          valid_sets=[data_train_lgb, data_eval_lgb], # 検証データの指定
                          )

        pred =model_lgb.predict(x_test)
        predicted.iloc[valid_index]=pred
        models.append(model_lgb)
    return models,predicted

In [ ]:
model_xg,predicted_xg=xg_kf(x,y)

In [ ]:
model_lgb,predicted_lgb=lgb_kf(x,y)

In [ ]:
# y_check_train=model_xg.predict(data_train_xg)*0.3+model_lgb.predict(x_train)*0.7
# y_check_test=model_xg.predict(data_eval_xg)*0.3+model_lgb.predict(x_test)*0.7
# y_check=predicted_lgb*0.7+predicted_xg*0.3

In [ ]:
# 相関係数作成
# y_train_df=pd.DataFrame(y_train)
# y_train_df['val']=y_check_train
# print(y_train_df.corr())

# y_test=pd.DataFrame(y)
# y_test['val']=y_check
# print(y_test.corr())

In [ ]:
# df_feature = pd.DataFrame(model_[0].feature_importance(importance_type = 'gain'),
#                            index=x.columns,columns=['importance'])
# df_feature = df_feature.sort_values('importance', ascending=False)

# plt.figure(figsize=(40, 12))
# sns.barplot(x=x.columns, y='importance', data=df_feature)
# plt.title(f'Head LightGBM Feature Importance')
# plt.tight_layout()
# plt.show()

In [ ]:
# # 相関係数作成
# y_train_df=pd.DataFrame(y_train)
# y_train_df['val']=model.predict(x_train)
# print(y_train_df.corr())

# y_test=pd.DataFrame(y_test)
# y_test['val']=model.predict(x_test)
# print(y_test.corr())

In [ ]:
def predict(prediction_df):
    X_predict = prediction_df.drop(['row_id'],axis=1)
    X_predict = X_predict.drop(v,axis=1)
    X_predict_xg = xgb.DMatrix(X_predict)
    
    d_pre=[]
    for i in model_xg:
        d_pre.append(i.predict((X_predict_xg)))
    predict_xg =sum(d_pre)/len(d_pre)
    
    d_pre=[]
    for i in model_lgb:
        d_pre.append(i.predict((X_predict)))
    predict_lgb =sum(d_pre)/len(d_pre)

    prediction = predict_xg*0.3+predict_lgb*0.7
    return prediction

In [ ]:
import ubiquant

env = ubiquant.make_env()  # initialize the environment
iter_test = env.iter_test()  # an iterator which loops over the test set and sample submission

#テストデータから不要列落とす

for (test_df, sample_prediction_df) in iter_test:
    #test_df.info()
    sample_prediction_df['target'] = predict(test_df) # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions